In [1]:
import utils
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from scipy.sparse import hstack
import pandas as pd
from skrub import TableReport
from jours_feries_france import JoursFeries
from vacances_scolaires_france import SchoolHolidayDates
import numpy as np

In [3]:
X, y, X_test = utils.get_and_process_data()

/users/eleves-a/2024/felix.brochier/Python_Data_Challenge-1/utils.py:45: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  closest_value = non_nan_values.iloc[(non_nan_values - value).abs().argmin()]
/users/eleves-a/2024/felix.brochier/Python_Data_Challenge-1/utils.py:45: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  closest_value = non_nan_values.iloc[(non_nan_values - value).abs().argmin()]
/users/eleves-a/2024/felix.brochier/Python_Data_Challenge-1/utils.py:194: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame o

In [12]:
TableReport(X)

Processing column  22 / 22


,,,,,,,,,,,,,,,,,,,,,,


In [23]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import VotingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.pipeline import Pipeline

# Define GPU-enabled regressors
xgb_regressor = XGBRegressor(tree_method='gpu_hist', gpu_id=0, random_state=42)
lgbm_regressor = LGBMRegressor(device='gpu', gpu_device_id=0, random_state=42)

# Define CPU-based Random Forest
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Columns to exclude from scaling
exclude_columns = ["is_weekend", "is_school_holiday", "road_work", "confinement", "couvre_feu"]
numerical_features = [
    col for col in X.columns if col not in exclude_columns + ["counter_name"]
]

# Preprocessing: One-Hot Encode "counter_name" and scale numerical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),  # Scale numerical columns
        ('cat', OneHotEncoder(), ['counter_name'])      # One-hot encode "counter_name"
    ],
    remainder='passthrough'  # Keep other columns unchanged
)

# Define the Voting Regressor
voting_regressor = VotingRegressor([
    ('xgb', xgb_regressor),
    ('lgbm', lgbm_regressor),
    ('rf', rf_regressor)
])

# Complete Pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),  # Preprocessing step
    ('voting_regressor', voting_regressor)  # Voting regressor
])

# # Fit the pipeline
pipeline.fit(X, y)

# # Make predictions
# predictions = pipeline.predict(X_test)


/users/eleves-a/2024/felix.brochier/Python_Data_Challenge-1/.venv/lib64/python3.9/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The XGBRegressor or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


AttributeError: 'super' object has no attribute '__sklearn_tags__'

In [ ]:
data = pd.read_parquet("data/train.parquet")
data = data.sort_values(["date", "counter_name"])
data_test = pd.read_parquet("data/final_test.parquet")
data_test = data_test.sort_values(["date", "counter_name"])


In [ ]:
external_conditions = pd.read_csv('data/external_data.csv')
external_conditions['date'] = pd.to_datetime(external_conditions['date'])

In [ ]:
# Step 1: Sort the `external_conditions` DataFrame by the `date` column
external_conditions = external_conditions.sort_values(by='date')

# Drop columns with more than 40% NaN values
threshold = len(external_conditions) * 0.4
external_conditions = external_conditions.dropna(thresh=threshold, axis=1)

# Step 2: Remove duplicate entries based on the `date` column
external_conditions = external_conditions.drop_duplicates(subset='date')

# Step 3: Convert the 'date' column to datetime
external_conditions['date'] = pd.to_datetime(external_conditions['date'])

# Step 4: Create a complete date range from the minimum to the maximum date in the DataFrame
date_range = pd.date_range(start=external_conditions['date'].min(), end=external_conditions['date'].max(), freq='H')

# Step 5: Create a DataFrame from the date_range
date_range_df = pd.DataFrame(date_range, columns=['date'])

# Step 6: Merge the date_range DataFrame with the external_conditions DataFrame on the 'date' column
full_external_conditions = pd.merge(date_range_df, external_conditions, on='date', how='left')

# Fonction qui fait ce qu'on voulait faire avec ffill et bfill mais a la place prends la valeur la plus proche
def fill_closest_value_all_columns(df):
    """Fill NaN values with the closest value for all numeric columns in the DataFrame."""
    filled_df = df.copy()
    
    for column in filled_df.columns:
        if filled_df[column].dtype.kind in 'biufc':  # Numeric columns
            non_nan_values = filled_df[column].dropna()
            
            def find_closest(value):
                if pd.isna(value):
                    closest_value = non_nan_values.iloc[(non_nan_values - value).abs().argmin()]
                    return closest_value
                return value
            
            filled_df[column] = filled_df[column].apply(find_closest)
    
    return filled_df

# Apply the function to the DataFrame
filled_external_conditions = fill_closest_value_all_columns(full_external_conditions)

In [ ]:
# Merge the DataFrames
merged_conditions = pd.merge(data, filled_external_conditions, on='date', how='left')

merged_conditions = utils._column_rename(merged_conditions)


merged_conditions_test = pd.merge(data_test, filled_external_conditions, on='date', how='left')

merged_conditions_test = utils._column_rename(merged_conditions_test)

In [ ]:
# Ensure "date" is in datetime format
merged_conditions["date"] = pd.to_datetime(merged_conditions["date"], errors="coerce")

# Drop rows with invalid datetime entries
df = merged_conditions.dropna(subset=["date"])

# Extract date and time features
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["weekday"] = df["date"].dt.dayofweek
df["day"] = df["date"].dt.day
df["hour"] = df["date"].dt.hour
df["is_weekend"] = (df["weekday"] >= 5).astype(int)

# Handle school and public holidays
unique_dates = df["date"].dt.date.unique()
d = SchoolHolidayDates()
f = JoursFeries()

try:
    dict_school_holidays = {date: d.is_holiday_for_zone(date, "C") for date in unique_dates}
    df["is_school_holiday"] = df["date"].dt.date.map(dict_school_holidays).fillna(0).astype(int)
except Exception as e:
    print(f"Error with school holidays mapping: {e}")
    df["is_school_holiday"] = 0

try:
    dict_public_holidays = {date: f.is_bank_holiday(date, zone="Métropole") for date in unique_dates}
    df["is_public_holiday"] = df["date"].dt.date.map(dict_public_holidays).fillna(0).astype(int)
except Exception as e:
    print(f"Error with public holidays mapping: {e}")
    df["is_public_holiday"] = 0

# Ensure "date" is in datetime format
merged_conditions_test["date"] = pd.to_datetime(merged_conditions_test["date"], errors="coerce")

# Drop rows with invalid datetime entries
df_test = merged_conditions_test.dropna(subset=["date"])

# Extract date and time features
df_test["year"] = df_test["date"].dt.year
df_test["month"] = df_test["date"].dt.month
df_test["weekday"] = df_test["date"].dt.dayofweek
df_test["day"] = df_test["date"].dt.day
df_test["hour"] = df_test["date"].dt.hour
df_test["is_weekend"] = (df_test["weekday"] >= 5).astype(int)

# Handle school and public holidays
unique_dates = df_test["date"].dt.date.unique()
d = SchoolHolidayDates()
f = JoursFeries()

try:
    dict_school_holidays = {date: d.is_holiday_for_zone(date, "C") for date in unique_dates}
    df_test["is_school_holiday"] = df_test["date"].dt.date.map(dict_school_holidays).fillna(0).astype(int)
except Exception as e:
    print(f"Error with school holidays mapping: {e}")
    df_test["is_school_holiday"] = 0

try:
    dict_public_holidays = {date: f.is_bank_holiday(date, zone="Métropole") for date in unique_dates}
    df_test["is_public_holiday"] = df_test["date"].dt.date.map(dict_public_holidays).fillna(0).astype(int)
except Exception as e:
    print(f"Error with public holidays mapping: {e}")
    df_test["is_public_holiday"] = 0

In [ ]:
start_date_Monpar = "2021-01-25"
end_date_Monpar = "2021-02-23"
start_date_Clichy_NO_SE = "2021-04-09"
end_date_Clichy = "2021-07-20"
start_date_Clichy_SE_NO = "2021-03-23"
start_date_Pompidou = "2021-03-13"
end_date_Pompidou = "2021-04-01"

df["road_work_Monpar_O_E"] = np.where(
    (data["date"] >= start_date_Monpar)
    & (data["date"] <= end_date_Monpar)
    & (data["counter_name"] == "152 boulevard du Montparnasse O-E"),
    1,
    0,
)
df["road_work_Monpar_E_O"] = np.where(
    (df["date"] >= start_date_Monpar)
    & (df["date"] <= end_date_Monpar)
    & (df["counter_name"] == "152 boulevard du Montparnasse E-O"),
    1,
    0,
)
df["road_work_Clichy_NO_SE"] = np.where(
    (df["date"] >= start_date_Clichy_NO_SE)
    & (df["date"] <= end_date_Clichy)
    & (df["counter_name"] == "20 Avenue de Clichy NO-SE"),
    1,
    0,
)
df["road_work_Clichy_SE_NO"] = np.where(
    (df["date"] >= start_date_Clichy_SE_NO)
    & (df["date"] <= end_date_Clichy)
    & (df["counter_name"] == "20 Avenue de Clichy SE-NO"),
    1,
    0,
)
df["road_work_Pompidou_NE_SO"] = np.where(
    (df["date"] >= start_date_Pompidou)
    & (df["date"] <= end_date_Pompidou)
    & (df["counter_name"] == "Voie Georges Pompidou NE-SO"),
    1,
    0,
)
df["road_work_Pompidou_SO_NE"] = np.where(
    (df["date"] >= start_date_Pompidou)
    & (df["date"] <= end_date_Pompidou)
    & (df["counter_name"] == "Voie Georges Pompidou SO-NE"),
    1,
    0,
)

df["road_work"] = (
    df["road_work_Monpar_E_O"]
    + df["road_work_Monpar_O_E"]
    + df["road_work_Clichy_NO_SE"]
    + df["road_work_Clichy_SE_NO"]
    + df["road_work_Pompidou_NE_SO"]
    + df["road_work_Pompidou_SO_NE"]
)
df.drop(
    [
        "road_work_Monpar_E_O",
        "road_work_Monpar_O_E",
        "road_work_Clichy_NO_SE",
        "road_work_Clichy_SE_NO",
        "road_work_Pompidou_NE_SO",
        "road_work_Pompidou_SO_NE",
    ],
    axis=1,
    inplace=True,
)

df["log_bike_count"][
    (df["date"] >= start_date_Monpar)
    & (df["date"] <= end_date_Monpar)
    & (df["counter_name"] == "152 boulevard du Montparnasse E-O")
] = 0
df["log_bike_count"][
    (df["date"] >= start_date_Monpar)
    & (df["date"] <= end_date_Monpar)
    & (df["counter_name"] == "152 boulevard du Montparnasse O-E")
] = 0
df["log_bike_count"][
    (df["date"] >= start_date_Clichy_NO_SE)
    & (df["date"] <= end_date_Clichy)
    & (df["counter_name"] == "20 Avenue de Clichy NO-SE")
] = 0
df["log_bike_count"][
    (df["date"] >= start_date_Clichy_SE_NO)
    & (df["date"] <= end_date_Clichy)
    & (df["counter_name"] == "20 Avenue de Clichy SE-NO")
] = 0
df["log_bike_count"][
    (df["date"] >= start_date_Pompidou)
    & (df["date"] <= end_date_Pompidou)
    & (df["counter_name"] == "Voie Georges Pompidou NE-SO")
] = 0
df["log_bike_count"][
    (df["date"] >= start_date_Pompidou)
    & (df["date"] <= end_date_Pompidou)
    & (df["counter_name"] == "Voie Georges Pompidou SO-NE")
] = 0

df_test['road_work'] = 0

In [ ]:
# Drop the columns from the df and df_test dataframes using the columns_to_drop list from the utils file
columns_to_drop = utils.columns_to_drop
df = df.drop(columns=columns_to_drop)
df_test = df_test.drop(columns=columns_to_drop)

In [ ]:
TableReport(df)

In [ ]:
TableReport(df_test)

## Test using flaml and the GPU

In [ ]:
import pandas as pd
from flaml import AutoML
from skrub import TableVectorizer


# Preprocess the dataset
X = df.drop(columns=['log_bike_count', 'bike_count'])
y = df['log_bike_count']

# Split the data into training and validation sets based on the last 10% of dates
validation_split_index = int(len(df) * 0.9)
X_train, X_val = X.iloc[:validation_split_index], X.iloc[validation_split_index:]
y_train, y_val = y.iloc[:validation_split_index], y.iloc[validation_split_index:]

# Initialize the TableVectorizer
vectorizer = TableVectorizer()

# Fit and transform the training data
X_train_transformed = vectorizer.fit_transform(X_train)
X_val_transformed = vectorizer.transform(X_val)



In [ ]:
import optuna
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from ngboost import NGBRegressor
from h2o.automl import H2OAutoML
import h2o

# Initialize H2O
h2o.init()

# Dictionary to store the best parameters for each model
best_params = {}

# Define the objective function for Optuna
def objective(trial):
    # Model selection
    model_name = trial.suggest_categorical("model", ["RandomForest", "NGBoost", "H2OAutoML"])
    
    if model_name == "RandomForest":
        n_estimators = trial.suggest_int("n_estimators", 50, 500)
        max_depth = trial.suggest_int("max_depth", 2, 32)
        min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42,
        )
        model.fit(X_train_transformed, y_train)
        y_pred = model.predict(X_val_transformed)
    
    elif model_name == "NGBoost":
        learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-1)
        n_estimators = trial.suggest_int("n_estimators", 50, 500)
        model = NGBRegressor(
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            random_state=42,
        )
        model.fit(X_train_transformed, y_train)
        y_pred = model.predict(X_val_transformed)
    
    elif model_name == "H2OAutoML":
        # Convert datasets to H2O frames
        train = h2o.H2OFrame(pd.concat([X_train, y_train], axis=1))
        val = h2o.H2OFrame(pd.concat([X_val, y_val], axis=1))
        
        # Specify predictors and response column
        predictors = X_train.columns.tolist()
        response = "log_bike_count"  # Update with your target column name
        
        # Run H2O AutoML
        automl = H2OAutoML(max_models=10, seed=42, nfolds=3)
        automl.train(x=predictors, y=response, training_frame=train)
        
        # Predict on validation set
        y_pred = automl.leader.predict(val).as_data_frame()["predict"].values

    # Compute the Mean Squared Error (MSE)
    mse = mean_squared_error(y_val, y_pred)
    return mse

# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Get the best trial and parameters
best_trial = study.best_trial
best_model_params = study.best_params
print("Best Trial:", best_trial)
print("Best Model Parameters:", best_model_params)

# Save the best model
model_name = best_model_params["model"]
if model_name == "H2OAutoML":
    # Save H2O AutoML model
    automl.leader.save_mojo(f"best_{model_name}.mojo")
    print(f"Best H2O AutoML model saved as 'best_{model_name}.mojo'")
else:
    # Save sklearn or NGBoost models
    joblib.dump(best_model, f"best_{model_name}.joblib")
    print(f"Best model saved as 'best_{model_name}.joblib'")

# Shut down H2O
h2o.shutdown(prompt=False)


In [24]:
from skrub import TableVectorizer
from xgboost import XGBRegressor
import optuna
from sklearn.model_selection import train_test_split

# Preprocess the dataset using TableVectorizer
X = df.drop(columns=['log_bike_count', 'bike_count'])
y = df['log_bike_count']

# Split the data into training and validation sets based on the last 10% of dates
validation_split_index = int(len(df) * 0.9)
X_train, X_val = X.iloc[:validation_split_index], X.iloc[validation_split_index:]
y_train, y_val = y.iloc[:validation_split_index], y.iloc[validation_split_index:]

# Initialize the TableVectorizer
vectorizer = TableVectorizer()

# Fit and transform the training data
X_train_transformed = vectorizer.fit_transform(X_train)
X_val_transformed = vectorizer.transform(X_val)


/users/eleves-a/2024/felix.brochier/Python_Data_Challenge-1/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'df' is not defined

## Tune XGBoost using Optuna hyperparameter

In [ ]:
import joblib
from xgboost import XGBRegressor
import optuna

# Define the objective function for Optuna
def objective(trial):
    param = {
        'objective': 'reg:squarederror',
        'n_estimators': trial.suggest_int('n_estimators', 200, 600),
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.2),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.9),
        'random_state': 42,
        'tree_method': 'gpu_hist',  # Enable GPU support
        'predictor': 'gpu_predictor'
    }
    model = XGBRegressor(**param)
    model.fit(X_train_transformed, y_train)
    return model.score(X_val_transformed, y_val)  # Maximizing validation R² score

# Create a study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Get the best parameters
best_params = study.best_params

# Add GPU-specific parameter to the best parameters
best_params['tree_method'] = 'gpu_hist'  # Ensure GPU is used for the final model

# Save the best parameters to a file using joblib
joblib.dump(best_params, 'xg_boost_best_params.pkl')

# Train the final model with the best parameters
X_transformed = vectorizer.transform(X)  # Transform the entire dataset
final_model = XGBRegressor(**best_params)
final_model.fit(X_transformed, y)


## Tune LGBM using Optuna 

In [26]:
# Split the data into training and validation sets based on the last 10% of dates
validation_split_index = int(len(X) * 0.9)
X_train, X_val = X.iloc[:validation_split_index], X.iloc[validation_split_index:]
y_train, y_val = y.iloc[:validation_split_index], y.iloc[validation_split_index:]

# Initialize the TableVectorizer
vectorizer = TableVectorizer()

# Fit and transform the training data
X_train_transformed = vectorizer.fit_transform(X_train)
X_val_transformed = vectorizer.transform(X_val)

/users/eleves-a/2024/felix.brochier/Python_Data_Challenge-1/.venv/lib64/python3.9/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The TableVectorizer or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


In [29]:
import joblib
from lightgbm import LGBMRegressor, early_stopping
import optuna
import pandas as pd

# Sanitize feature names
def sanitize_feature_names(df):
    """Sanitize feature names to remove special JSON characters."""
    df.columns = df.columns.str.replace(r"[^\w\d_]", "_", regex=True)
    return df

# Sanitize training and validation feature names
X_train_transformed = sanitize_feature_names(pd.DataFrame(X_train_transformed))
X_val_transformed = sanitize_feature_names(pd.DataFrame(X_val_transformed))

# Define the objective function for Optuna
def objective(trial):
    param = {
        'objective': 'regression',
        'boosting_type': 'gbdt',
        'n_estimators': trial.suggest_int('n_estimators', 200, 600),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', -1, 20),  # -1 means no limit
        'num_leaves': trial.suggest_int('num_leaves', 31, 256),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'random_state': 42,
        'device': 'gpu',  # Enable GPU support
        'gpu_device_id': 0  # Use the first GPU device
    }
    model = LGBMRegressor(**param)
    model.fit(
        X_train_transformed, y_train,
        eval_set=[(X_val_transformed, y_val)],
        eval_metric='rmse',
        callbacks=[early_stopping(stopping_rounds=20)]
    )
    return model.best_score_['valid_0']['rmse']  # Minimize RMSE

# Create a study and optimize the objective function
study = optuna.create_study(direction='minimize')  # Minimizing RMSE
study.optimize(objective, n_trials=50)

# Get the best parameters
best_params = study.best_params

# Add GPU-specific parameter to the best parameters
best_params['device'] = 'gpu'  # Ensure GPU is used for the final model

# Save the best parameters to a file using joblib
joblib.dump(best_params, 'lightgbm_best_params.pkl')

# Train the final model with the best parameters
X_transformed = sanitize_feature_names(pd.DataFrame(vectorizer.transform(X)))  # Transform the entire dataset
final_model = LGBMRegressor(**best_params)
final_model.fit(X_transformed, y)


[I 2024-12-10 16:09:28,238] A new study created in memory with name: no-name-0b64e342-7b3c-4600-a431-959cf57a0b8c


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2828
[LightGBM] [Info] Number of data points in the train set: 447144, number of used features: 51
[LightGBM] [Info] Using GPU Device: NVIDIA RTX A5000, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 44 dense feature groups (18.76 MB) transferred to GPU in 0.014667 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 3.042398
Training until validation scores don't improve for 20 rounds


[I 2024-12-10 16:09:45,820] Trial 0 finished with value: 0.45359378700184444 and parameters: {'n_estimators': 369, 'learning_rate': 0.2208340478880343, 'max_depth': 18, 'num_leaves': 139, 'subsample': 0.830511140506035, 'colsample_bytree': 0.6800110619273798}. Best is trial 0 with value: 0.45359378700184444.


...

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2828
[LightGBM] [Info] Number of data points in the train set: 447144, number of used features: 51
[LightGBM] [Info] Using GPU Device: NVIDIA RTX A5000, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 44 dense feature groups (18.76 MB) transferred to GPU in 0.014302 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 3.042398
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

[I 2024-12-10 16:20:31,475] Trial 49 finished with value: 0.4949930214792919 and parameters: {'n_estimators': 385, 'learning_rate': 0.029275960993832596, 'max_depth': 6, 'num_leaves': 194, 'subsample': 0.5457311139005518, 'colsample_bytree': 0.7592413542395129}. Best is trial 34 with value: 0.4279859221494695.
/users/eleves-a/2024/felix.brochier/Python_Data_Challenge-1/.venv/lib64/python3.9/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The TableVectorizer or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2830
[LightGBM] [Info] Number of data points in the train set: 496827, number of used features: 51
[LightGBM] [Info] Using GPU Device: NVIDIA RTX A5000, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 44 dense feature groups (20.85 MB) transferred to GPU in 0.016656 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 3.075693
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

/users/eleves-a/2024/felix.brochier/Python_Data_Challenge-1/.venv/lib64/python3.9/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The LGBMRegressor or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


LGBMRegressor(colsample_bytree=0.6293816639977194, device='gpu',
              learning_rate=0.06096880786868278, max_depth=10, n_estimators=356,
              num_leaves=203, subsample=0.8389519986730779)

In [32]:
import joblib
from lightgbm import LGBMRegressor
from skrub import TableVectorizer

# Function to sanitize feature names
def sanitize_feature_names(df):
    """Sanitize feature names to remove special JSON characters."""
    df.columns = df.columns.str.replace(r"[^\w\d_]", "_", regex=True)
    return df

# Load the best parameters from the pickle file
best_params = joblib.load('lightgbm_best_params.pkl')

# Initialize the TableVectorizer
vectorizer = TableVectorizer()

# Fit and transform the data
X_transformed = vectorizer.fit_transform(X)

# Sanitize feature names in the transformed data
X_transformed = sanitize_feature_names(X_transformed)

# Train the final model with the best parameters
final_model = LGBMRegressor(**best_params)
final_model.fit(X_transformed, y)

# Print model parameters
print("Trained model parameters:")
print(final_model.get_params())


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2839
[LightGBM] [Info] Number of data points in the train set: 496827, number of used features: 51
[LightGBM] [Info] Using GPU Device: NVIDIA RTX A5000, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 44 dense feature groups (20.85 MB) transferred to GPU in 0.024426 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 3.075693
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Trained model parameters:
{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.6293816639977194, 'importance

In [34]:
# Transform the test data using the same vectorizer instance
X_test_transformed = vectorizer.transform(X_test)

# Make predictions
y_pred = final_model.predict(X_test_transformed)

print("Predictions:", y_pred)


/users/eleves-a/2024/felix.brochier/Python_Data_Challenge-1/.venv/lib64/python3.9/site-packages/sklearn/utils/_tags.py:354: FutureWarning: The TableVectorizer or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


Predictions: [1.45958483 1.30358823 1.26122939 ... 3.23660304 2.57403481 2.58017658]


In [36]:
df_submission = pd.DataFrame(y_pred, columns=["log_bike_count"])
df_submission.index = X_test.index
df_submission.index.name = "Id"
df_submission.to_csv("test_pipeline_lgbm.csv", index=True)

In [ ]:
test_data = pd.read_parquet('data/final_test.parquet')
# Merge the DataFrames
merged_conditions = pd.merge(test_data, filled_external_conditions, on='date', how='left')

merged_conditions = utils._column_rename(merged_conditions)

# Ensure "date" is in datetime format
merged_conditions["date"] = pd.to_datetime(merged_conditions["date"], errors="coerce")

# Drop rows with invalid datetime entries
df_test = merged_conditions.dropna(subset=["date"])

# Extract date and time features
df_test["year"] = df_test["date"].dt.year
df_test["month"] = df_test["date"].dt.month
df_test["weekday"] = df_test["date"].dt.dayofweek
df_test["day"] = df_test["date"].dt.day
df_test["hour"] = df_test["date"].dt.hour
df_test["is_weekend"] = (df_test["weekday"] >= 5).astype(int)

# Handle school and public holidays
unique_dates_test = df_test["date"].dt.date.unique()

try:
    dict_school_holidays_test = {date: d.is_holiday_for_zone(date, "C") for date in unique_dates_test}
    df_test["is_school_holiday"] = df_test["date"].dt.date.map(dict_school_holidays_test).fillna(0).astype(int)
except Exception as e:
    print(f"Error with school holidays mapping: {e}")
    df_test["is_school_holiday"] = 0

try:
    dict_public_holidays_test = {date: f.is_bank_holiday(date, zone="Métropole") for date in unique_dates_test}
    df_test["is_public_holiday"] = df_test["date"].dt.date.map(dict_public_holidays_test).fillna(0).astype(int)
except Exception as e:
    print(f"Error with public holidays mapping: {e}")
    df_test["is_public_holiday"] = 0

# Predict using the pipeline
y_pred_test = pipeline.predict(df_test)

In [ ]:
df_submission = pd.DataFrame(y_pred_test, columns=["log_bike_count"])
df_submission.index.name = "Id"
df_submission
df_submission.to_csv("/Users/felix/Documents/X/Cours Python/Kaggle/submission/test_pipeline.csv", index=True)

## En dessous c'est des tests d'avant ca ne fait pas tourner ce qui marche actuellement

In [ ]:
# Add the new category to categorical columns
for col in df.select_dtypes(include=['category']).columns:
	df[col] = df[col].cat.add_categories([0])

# Fill NaN values with 0
df = df.fillna(0)

In [ ]:
y_train = df['log_bike_count'].values
X_train = df.drop(['log_bike_count', "bike_count"], axis=1)

date_cols = ["year", "month", "weekday", "day", "hour", "is_weekend", "is_school_holiday", "is_public_holiday"]
categorical_cols = ["counter_name"]
numerical_cols = [
    'latitude', 'longitude', 'Sea Level Pressure (hPa)', 'Pressure Tendency (hPa/3h)',
    'Pressure Tendency Code', 'Wind Direction (°)', 'Wind Speed (m/s)', 'Air Temperature (°C)',
    'Dew Point Temperature (°C)', 'Relative Humidity (%)', 'Visibility (m)', 'Present Weather Code',
    'Past Weather Code 1', 'Past Weather Code 2', 'Total Cloud Cover (oktas)', 'Cloud Base Height (m)',
    'Lowest Cloud Base Height (m)', 'Low Cloud Type', 'Station Level Pressure (hPa)', '24h Pressure Tendency (hPa)',
    '10min Max Wind Gust (m/s)', 'Max Wind Gust (m/s)', 'Measurement Period Duration', 'Ground State',
    'Snow Height (cm)', 'New Snow Depth (cm)', 'New Snowfall Duration (hours)', 'Rainfall (1h, mm)',
    'Rainfall (3h, mm)', 'Rainfall (6h, mm)', 'Rainfall (12h, mm)', 'Rainfall (24h, mm)',
    'Layer 1 Cloud Cover (oktas)', 'Layer 1 Cloud Type', 'Layer 1 Cloud Base Height (m)'
]


# 1. Apply column transformations
# One-hot encode date columns
date_encoder = OneHotEncoder(handle_unknown="ignore")
date_encoded = date_encoder.fit_transform(X_train[date_cols])

# One-hot encode categorical columns
cat_encoder = OneHotEncoder(handle_unknown="ignore")
cat_encoded = cat_encoder.fit_transform(X_train[categorical_cols])

# Standard scale numerical columns
num_scaler = StandardScaler()
num_scaled = num_scaler.fit_transform(X_train[numerical_cols])

X_transformed = hstack([date_encoded, cat_encoded, num_scaled]).toarray()

# 2. Train the model
model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model.fit(X_transformed, y_train)



In [ ]:
X_test = utils.get_test_data()

In [ ]:
# Merge the DataFrames
merged_conditions = pd.merge(X_test, external_conditions, on='date', how='left')

merged_conditions = utils._column_rename(merged_conditions)
# Ensure "date" is in datetime format
merged_conditions["date"] = pd.to_datetime(merged_conditions["date"], errors="coerce")

# Drop rows with invalid datetime entries
df = merged_conditions.dropna(subset=["date"])

# Extract date and time features
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["weekday"] = df["date"].dt.dayofweek
df["day"] = df["date"].dt.day
df["hour"] = df["date"].dt.hour
df["is_weekend"] = (df["weekday"] >= 5).astype(int)

# Handle school and public holidays
unique_dates = df["date"].dt.date.unique()
d = SchoolHolidayDates()
f = JoursFeries()

try:
    dict_school_holidays = {date: d.is_holiday_for_zone(date, "C") for date in unique_dates}
    df["is_school_holiday"] = df["date"].dt.date.map(dict_school_holidays).fillna(0).astype(int)
except Exception as e:
    print(f"Error with school holidays mapping: {e}")
    df["is_school_holiday"] = 0

try:
    dict_public_holidays = {date: f.is_bank_holiday(date, zone="Métropole") for date in unique_dates}
    df["is_public_holiday"] = df["date"].dt.date.map(dict_public_holidays).fillna(0).astype(int)
except Exception as e:
    print(f"Error with public holidays mapping: {e}")
    df["is_public_holiday"] = 0

In [ ]:
# Process the test data with the same transformations as the training data
# 1. Apply column transformations
# One-hot encode date columns
date_encoded_test = date_encoder.transform(df[date_cols])

# One-hot encode categorical columns
cat_encoded_test = cat_encoder.transform(df[categorical_cols])

# Standard scale numerical columns
num_scaled_test = num_scaler.transform(df[numerical_cols])

# Combine all transformed features
X_test_transformed_numeric = hstack([date_encoded_test, cat_encoded_test, num_scaled_test]).toarray()

# 2. Make predictions
y_pred = model.predict(X_test_transformed_numeric)


In [ ]:
df_submission = pd.DataFrame(y_pred, columns=["log_bike_count"])
df_submission.index.name = "Id"

In [ ]:
df_submission.to_csv("/Users/felix/Documents/X/Cours Python/Kaggle/submission/test_pipeline.csv", index=True)